# Training model

In [1]:
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import inchi
from models.mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from statistics import mean, stdev
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
# Custom loss function to handle missing values in the input data
def mse(y_pred, y_true):
    y_true = torch.where(torch.isnan(y_true), y_pred, y_true)
    cost = torch.abs(y_pred - y_true)
    cost = torch.sum(cost,dim=-1)
    cost = cost**2
    cost = torch.mean(cost)/2
    return cost

class CustomModel_1(nn.Module):
    def __init__(self):
        super(CustomModel_1, self).__init__()
        self.avalon_fc1 = nn.Linear(input_dim, 2000)
        self.avalon_fc2 = nn.Linear(2000, 700)
        self.avalon_fc3 = nn.Linear(700, 500)
        self.avalon_fc4 = nn.Linear(500, 100)

        self.mol2vec_fc1 = nn.Linear(300,100)
        self.dropout = nn.Dropout(p=0.2)
        
        self.concat_layer = nn.Linear(200,59)
        
        
    def forward(self, x):
        avalon_x = torch.relu(self.avalon_fc1(x[:,0:1024]))
        avalon_x = torch.relu(self.avalon_fc2(avalon_x))
        avalon_x = torch.relu(self.avalon_fc3(avalon_x))
        avalon_x = torch.relu(self.avalon_fc4(avalon_x))
        
        mol2vec_x = torch.relu(self.mol2vec_fc1(x[:,1024:]))
        mol2vec_x = self.dropout(mol2vec_x)
        
        final = self.concat_layer(torch.cat((avalon_x,mol2vec_x),dim=1))
        
        return final

class CustomModel_2(nn.Module):
    def __init__(self):
        super(CustomModel_2, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1500)
        self.fc2 = nn.Linear(1500, 500)
        self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(100, 59)
        
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

In [3]:
# Set random seed for reproducibility
seed = 7
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device('cuda:6') if torch.cuda.is_available() else torch.device('cpu')

# 경로 설정
path = os.getcwd()
model_path = path + '/models/'
if not os.path.exists(model_path):
    os.makedirs(model_path)
data_path = path + '/data/'
random_split = data_path + '/random_split'
if not os.path.exists(data_path):
    os.makedirs(data_path)
if not os.path.exists(random_split):
    os.makedirs(random_split)

# Load training dataset
avalon = pd.read_csv(data_path + "Avalon_bits.csv")
mol2vec = pd.read_csv(data_path + "Mol2vec.csv")
ld50 = pd.read_csv(data_path + "dataset.csv")

mol2vec_df = pd.concat([ld50.iloc[:,0:1],avalon.iloc[:,2:],mol2vec.iloc[:,2:],ld50.iloc[:,2:]],axis=1)


# kfold start
kfold=KFold(n_splits=5, shuffle=True, random_state=seed)
fold = 0
for train_indices, test_index in kfold.split(mol2vec_df):
    validation_indices, test_indices = train_test_split(test_index,test_size = 0.5, random_state=seed)
    fold += 1
    print(f'-------------------------------Fold {fold}-------------------------------')
    print('Load data')
    print(f'train num : {len(train_indices)}')
    print(f'val num : {len(validation_indices)}')
    print(f'test num : {len(test_indices)}')
    
    tasks = 59 # task 개수
    task_index = 1325 # model input과 task를 나누기 위한 index 값
    print('tasks: %s' % tasks)
    X_train = mol2vec_df.iloc[train_indices,1:task_index].values
    y_train = mol2vec_df.iloc[train_indices,task_index:].values
    X_val = mol2vec_df.iloc[validation_indices,1:task_index].values
    y_val = mol2vec_df.iloc[validation_indices,task_index:].values
    X_test = mol2vec_df.iloc[test_indices,1:task_index].values
    y_test = mol2vec_df.iloc[test_indices,task_index:].values
    
    mol2vec.iloc[train_indices,0:2].to_csv(f'{random_split}' + f'/train_fold_{fold}.csv',index=False)
    mol2vec.iloc[validation_indices,0:2].to_csv(f'{random_split}' + f'/validation_fold_{fold}.csv',index=False)
    mol2vec.iloc[test_indices,0:2].to_csv(f'{random_split}' + f'/test_fold_{fold}.csv',index=False)
    
    # Load test dataset
    df_test = pd.concat([mol2vec_df.iloc[val_indices,0:1],mol2vec_df.iloc[val_indices,task_index:]],axis=1)
    
    task_list = list(mol2vec_df.iloc[:,task_index:].columns)

    input_dim = X_train.shape[1]

    model = CustomModel_2().to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = mse


    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Training loop
    epochs = 50
    batch_size = 128 

    print('Training start')
    best_val_loss = 555555555555555555
    for epoch in range(epochs):
        train_loss = 0
        for i in range(0, len(X_train_tensor), batch_size):
            optimizer.zero_grad()
            train_batch_x, train_batch_y = X_train_tensor[i:i+batch_size], y_train_tensor[i:i+batch_size]
            outputs = model(train_batch_x)
            loss = criterion(outputs, train_batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        val_loss = 0
        for i in range(0, len(X_val_tensor), batch_size):
            model.eval()
            with torch.no_grad():
                val_batch_x, val_batch_y = X_val_tensor[i:i+batch_size], y_val_tensor[i:i+batch_size]
                val_outputs = model(val_batch_x)
                val_loss += criterion(val_outputs,val_batch_y).item()
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path + f'Best_model_fold_{fold}')    
        print(f'Epoch : {epoch}, Train loss : {train_loss}, Val_loss : {val_loss}')
    # Evaluation
    print('Prediction start')
    model = CustomModel_2()
    model.load_state_dict(torch.load(model_path + f'Best_model_fold_{fold}'))
    
    model.eval()
    with torch.no_grad():
        y_pred = model(X_val_tensor).numpy()

    # Transform predictions to DataFrame
    df_pred = pd.DataFrame(data=y_pred, columns=task_list)
    rtecs_ids = df_test['RTECS_ID'].values
    df_pred = df_pred.assign(RTECS_ID=rtecs_ids)

    # Reshape df_test and df_pred
    df_test = pd.melt(df_test, id_vars='RTECS_ID', value_vars=task_list, var_name='Task', value_name='LD50')
    df_pred = pd.melt(df_pred, id_vars='RTECS_ID', value_vars=task_list, var_name='Task', value_name='pred_LD50')

    # Merge df_test and df_pred
    final_df = pd.merge(df_test, df_pred,  how='left', left_on=['RTECS_ID','Task'], right_on = ['RTECS_ID','Task'])
    final_df = final_df.dropna(subset=['LD50'])

    final_df.to_csv(f"/home/psy/LD50/results/mol2vec_fold_{fold}.csv", index=False)

    y_true = final_df['LD50'].values
    y_pred = final_df['pred_LD50'].values

    # Overall performance
    print("\nOverall performance")
    print("Metric\tValue")
    print("r^2\t%.2f" % r2_score(y_true, y_pred))
    print("mae\t%.2f" % mean_absolute_error(y_true, y_pred))
    print("mse\t%.2f" % mean_squared_error(y_true, y_pred))
    print("rmse\t%.2f" % sqrt(mean_squared_error(y_true, y_pred)))

    # Calculate performance task-wise
    r2_scores = []
    mae_scores = []
    mse_scores = []
    rmse_scores = []

    for task, dft in final_df.groupby('Task'):
        y_true = dft['LD50'].values
        y_pred = dft['pred_LD50'].values

        r2_scores.append(r2_score(y_true, y_pred))
        mae_scores.append(mean_absolute_error(y_true, y_pred))
        mse_scores.append(mean_squared_error(y_true, y_pred))
        rmse_scores.append(sqrt(mean_squared_error(y_true, y_pred)))

    # Task-wise performance
    print("\nTask-wise performance")
    print("Metric\tAvg\tStdev")
    print("r^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("mae\t%.2")

/tmp/ipykernel_23005/2230538016.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  avalon = pd.read_csv(data_path + "Avalon_bits.csv")
/tmp/ipykernel_23005/2230538016.py:21: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  mol2vec = pd.read_csv(data_path + "Mol2vec.csv")


-------------------------------Fold 1-------------------------------
Load data
train num : 64064
val num : 8008
test num : 8009
tasks: 59
Training start
Epoch : 0, Train loss : 2269.2004393041134, Val_loss : 136.80580353736877
Epoch : 1, Train loss : 855.944313108921, Val_loss : 98.42281371355057
Epoch : 2, Train loss : 653.9289003014565, Val_loss : 97.49839067459106
Epoch : 3, Train loss : 543.1609357595444, Val_loss : 82.4841600060463
Epoch : 4, Train loss : 487.26026195287704, Val_loss : 81.52028003334999
Epoch : 5, Train loss : 434.9582031071186, Val_loss : 78.4399865269661
Epoch : 6, Train loss : 408.2961251437664, Val_loss : 80.10769954323769
Epoch : 7, Train loss : 379.38157215714455, Val_loss : 80.11142924427986
Epoch : 8, Train loss : 381.76974272727966, Val_loss : 84.5760890841484
Epoch : 9, Train loss : 341.64926090836525, Val_loss : 79.21719041466713
Epoch : 10, Train loss : 326.31132151186466, Val_loss : 68.31574723124504
Epoch : 11, Train loss : 340.31405948102474, Val_lo

Epoch : 2, Train loss : 624.8158215880394, Val_loss : 115.12195289134979
Epoch : 3, Train loss : 536.7242231667042, Val_loss : 98.97653409838676
Epoch : 4, Train loss : 481.3764029443264, Val_loss : 90.39678791165352
Epoch : 5, Train loss : 462.2240078896284, Val_loss : 104.94420635700226
Epoch : 6, Train loss : 417.2012409865856, Val_loss : 105.13534957170486
Epoch : 7, Train loss : 397.6802727431059, Val_loss : 108.99419951438904
Epoch : 8, Train loss : 376.54260228574276, Val_loss : 83.74887454509735
Epoch : 9, Train loss : 391.625191539526, Val_loss : 88.05699929594994
Epoch : 10, Train loss : 344.4292309731245, Val_loss : 69.37578082084656
Epoch : 11, Train loss : 342.8526588231325, Val_loss : 72.36282858252525
Epoch : 12, Train loss : 320.16066275537014, Val_loss : 65.4458821117878
Epoch : 13, Train loss : 318.69737058877945, Val_loss : 77.29242485761642
Epoch : 14, Train loss : 286.83479341864586, Val_loss : 66.43779090046883
Epoch : 15, Train loss : 271.53682585060596, Val_loss

Epoch : 6, Train loss : 385.6912610977888, Val_loss : 67.36844225227833
Epoch : 7, Train loss : 373.54804991185665, Val_loss : 67.1430436372757
Epoch : 8, Train loss : 350.3656442910433, Val_loss : 68.58828163146973
Epoch : 9, Train loss : 345.2431185692549, Val_loss : 68.10438817739487
Epoch : 10, Train loss : 350.4907045662403, Val_loss : 65.1615991294384
Epoch : 11, Train loss : 318.1879439353943, Val_loss : 65.85319557785988
Epoch : 12, Train loss : 304.21188339591026, Val_loss : 63.28145369887352
Epoch : 13, Train loss : 310.60490445792675, Val_loss : 65.95252834260464
Epoch : 14, Train loss : 270.56545862555504, Val_loss : 63.20058435201645
Epoch : 15, Train loss : 285.7966040968895, Val_loss : 62.682000398635864
Epoch : 16, Train loss : 262.12574565410614, Val_loss : 61.604928240180016
Epoch : 17, Train loss : 257.52628783881664, Val_loss : 70.21283859014511
Epoch : 18, Train loss : 248.1610263288021, Val_loss : 60.822383388876915
Epoch : 19, Train loss : 254.3688156157732, Val_

# Results of test_fold_1

In [4]:
import os
import pandas as pd
import numpy as np

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors

from models.mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from models.mol2vec.helpers import depict_identifier, mol_to_svg
from gensim.models import word2vec

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from statistics import mean, stdev
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [5]:
class CustomModel_2(nn.Module):
    def __init__(self):
        super(CustomModel_2, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1500)
        self.fc2 = nn.Linear(1500, 500)
        self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(100, 59)
        
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
    
def toxic_label(value):
    if value >= 5000: 
        return 'Safe_chemicals'
    elif value >= 500:
        return 'Slightly_toxic'
    elif value >= 50:
        return 'Moderately_toxic'
    else:
        return 'Highest_toxic'

In [9]:
# 경로 설정
path = os.getcwd()
model_path = path + '/models/'
if not os.path.exists(model_path):
    os.makedirs(model_path)
data_path = path + '/data/'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
device = torch.device('cuda:6') if torch.cuda.is_available() else torch.device('cpu')

# reverse standardization을 위한 MolWt 추출
data = pd.read_csv("/home/psy/LD50/Toxicity_prediction/data/random_split/test_fold_1.csv")
smiles_list = data['SMILES'].values # SMILES 부분민 추출
mol = [ Chem.MolFromSmiles(smiles) for smiles in smiles_list]
molecular_weight = [Descriptors.MolWt(mols) for mols in mol]
data['MolWt'] = molecular_weight

# SMILES 문자열을 Molecule 객체로 변환하고 Avalon fingerprint 생성
avalon_fps = []
for smiles in smiles_list: # 각각의 smile에 해당하는 avalon_fp 를 얻기 위한 for문
    mol = Chem.MolFromSmiles(smiles) # smiles를 molecule로 변환
    if mol is not None:
        avalon_fp = rdkit.Avalon.pyAvalonTools.GetAvalonFP(mol, nBits=1024) # molecule 을 avalon_fp로 변환
        binary_avalon_fp = avalon_fp.ToList() # 0,1 로 구성된 1024bit로 표현
        avalon_fps.append(binary_avalon_fp)
    else:
        print(f"Failed to generate Avalon fingerprint for SMILES: {smiles}") # SMILES 정보가 database에 존재하지 않을 시 error
avalon_fps = np.array(avalon_fps)

# avalon_fp의 각 col name을 Avalon_i 로 표현
col_list = [] 
for i in range(1024):
    col_names = f'Avalon_{i}'
    col_list.append(col_names)
    
avalon_bits = pd.DataFrame(avalon_fps)
avalon_bits.columns = col_list

# Unseen data의 mo2lvec embedding vector 생성
mol2vec_df = data.iloc[:,0:2] # ID와 smiles만 추출

mol = [Chem.MolFromSmiles(x) for x in smiles_list]
#Draw.MolsToGridImage(mol, molsPerRow=5, useSVG=False) # 시각화
mol2vec_df['ROMol'] = mol

# molecule 별로 sentence 생성
mol2vec_df['sentence'] = mol2vec_df.apply(lambda x: MolSentence(mol2alt_sentence(x['ROMol'], 1)), axis=1)

# pretrained mol2vec model 불러오기
model = word2vec.Word2Vec.load(model_path + 'mol2vec/mol2vec_300dim.pkl')

# mol2vec embedding vector 생성
mol2vec_df['mol2vec'] = [DfVec(x) for x in sentences2vec(mol2vec_df['sentence'], model, unseen='UNK')]

# mol2vec embedding vector 저장
mol2vec_emb = np.array([x.vec for x in mol2vec_df['mol2vec']])

col_list = [] 
for i in range(300):
    col_names = f'Mol2vec_{i}'
    col_list.append(col_names)
    
Mol2vec = pd.DataFrame(mol2vec_emb)
Mol2vec.columns = col_list

# prediction 결과를 reverse standardization 하여 class_assign을 하고 toxicity count와 percantage, score를 구하는 코드
# remove_col = data.columns[2:1085]
# data = data.drop(columns=remove_col)

test_set = pd.concat([data,avalon_bits,Mol2vec],axis=1)
X_test = test_set.iloc[:,3:].values
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
input_dim = X_test.shape[1]

model = CustomModel_2().to(device)
model.load_state_dict(torch.load(model_path + f'Best_model_fold_1'))

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cpu().numpy()
    
# task_list = remove_col[0:59]
pred = pd.DataFrame(y_pred)
# pred.columns = task_list
pred['molwt'] = data['MolWt'].values

for i in range(len(pred)):
    reverse_standardization = (1/10**(pred.iloc[i,0:59]))*pred.iloc[i,59]*10**(3)
    pred.iloc[i,0:59] = reverse_standardization
    
pred_label = pred.iloc[:,0:-1]
pred_label = pred_label.applymap(toxic_label)
pred_label = pd.concat([data,pred_label],axis=1)
count_label = pred_label.iloc[:,3:].T

pred_label['Safe_chemicals'] = 0
pred_label['Slightly_toxicity'] = 0
pred_label['Moderately_toxicity'] = 0
pred_label['Highest_toxicity'] = 0

count_list = []
for i in range(len(count_label.columns)):
    dict = count_label.iloc[:,i].value_counts().to_dict()
    count_list.append(dict)

for i in range(len(count_label.columns)):
    try:
        pred_label.loc[i,'Safe_chemicals'] = count_list[i]['Safe_chemicals']
    except KeyError:
        pred_label.loc[i,'Safe_chemicals'] = 0
    try:
        pred_label.loc[i,'Slightly_toxicity'] = count_list[i]['Slightly_toxic']
    except KeyError:
        pred_label.loc[i,'Slightly_toxicity'] = 0
    try:
        pred_label.loc[i,'Moderately_toxicity'] = count_list[i]['Moderately_toxic']
    except KeyError:
        pred_label.loc[i,'Moderately_toxicity'] = 0
    try:
        pred_label.loc[i,'Highest_toxicity'] = count_list[i]['Highest_toxic']
    except KeyError:
        pred_label.loc[i,'Highest_toxicity'] = 0
        
toxic_count = pred_label.iloc[:,-4:].values
toxic_percentage = np.round((toxic_count/59)*100,2)

col_names = ['Safe_chemicals_per','Slightly_toxicity_per','Moderately_toxicity_per','Highest_toxicity_per']
toxic_per = pd.DataFrame(toxic_percentage,columns = col_names)

results = pd.concat([pred_label.iloc[:,0:2],pred_label.iloc[:,-4:],toxic_per],axis=1)
results['toxic_score'] = np.round((results['Safe_chemicals']*0 + results['Slightly_toxicity']*3.33 + results['Moderately_toxicity']*6.66 + results['Highest_toxicity']*10)/59,2)
results['std'] = np.round(results.iloc[:,2:6].std(axis=1),2)

results.to_csv(path+'/Results/Prediction_of_toxicity_with_test_fold_1.csv',index=False)